In [ ]:
import os
import sys
import time

import numpy as np
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torchvision import models

In [ ]:
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_imagenet = torchvision.datasets.Imagenette(
    root="./Imagenette",
    download=True,
    split="val",
    transform=transform_val
)
test_loader = DataLoader(test_imagenet, batch_size=32, shuffle=False)

NameError: name 'transforms' is not defined

In [ ]:
imagenette_class_to_imagenet_idx = {
    0: 0,    # tench
    1: 6,    # English springer
    2: 13,   # cassette player
    3: 20,   # chain saw
    4: 23,   # church
    5: 28,   # French horn
    6: 33,   # garbage truck
    7: 42,   # gas pump
    8: 45,   # golf ball
    9: 53    # parachute
}
imagenet_idx_to_imagenette_class = {v: k for k, v in imagenette_class_to_imagenet_idx.items()}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50_weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights = resnet50_weights)
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:


# Evaluation function
def evaluate_model(model, dataloader, device, class_map):
    model.eval()  # eval mode
    correct = 0
    total = 0
    with torch.no_grad():  # no grad para eval
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # fw
            outputs = model(images)

            # pred class
            _, predicted = torch.max(outputs, 1)

            # map predicted ImageNet classes to Imagenette classes
            mapped_predictions = torch.tensor([class_map.get(p.item(), -1) for p in predicted]).to(device)

            # sacar predicciones inválidas
            valid_mask = (mapped_predictions != -1)
            mapped_predictions = mapped_predictions[valid_mask]
            labels = labels[valid_mask]

            # calcular pred correctas
            total += labels.size(0)
            correct += (mapped_predictions == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

In [ ]:
evaluate_model(model, test_loader, device, imagenet_idx_to_imagenette_class)

Accuracy: 100.00%


In [ ]:
model2 = models.vgg16(pretrained=True)
model2 = model2.to(device)
evaluate_model(model2, test_loader, device, imagenet_idx_to_imagenette_class)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy: 100.00%
